# APIs and SQL Joins Lab

The city of San Francisco wants to assess the quality of restaurants in the city. Their data is scattered across multiple sources and incomplete.

They tasked you to help them assess it.

They would like to know what the most common violations are where they happen most frequently.

## 1. Initial data inspection

To answer the question we will need to retrieve and merge data from multiple files.

Yelp provides data on food quality, that can be found at [this address](http://www.yelp.com/healthscores/feeds). We already downloaded 4 files that you can find in the [assets folder](../../assets/datasets/yelp/).

In the bonus part we will also use the Google Geocoding API and data on [Neighborhoods](https://www.google.com/fusiontables/DataSource?docid=1zNwsvTwj-dH0QxuuDrKFsyfNklajd7WwEyaZ2U9M#rows:id=1).

1. Open each of the files and inspect them visually
- What information do they contain?

In [3]:
business = pd.read_csv('/home/llevin/Desktop/DSI-SF-2-llevin16/Week 6 Notes & Code/Datasets/businesses.csv')
inspections = pd.read_csv('/home/llevin/Desktop/DSI-SF-2-llevin16/Week 6 Notes & Code/Datasets/inspections.csv')
legend = pd.read_csv('/home/llevin/Desktop/DSI-SF-2-llevin16/Week 6 Notes & Code/Datasets/legend.csv')
violations = pd.read_csv('/home/llevin/Desktop/DSI-SF-2-llevin16/Week 6 Notes & Code/Datasets/violations.csv')

In [4]:
violations.head(3).T

,0,1,2
business_id,10,10,10
date,20140729,20140729,20140114
description,Insufficient hot water or running water [ dat...,Unapproved or unmaintained equipment or utensi...,Inadequate and inaccessible handwashing facili...


In [5]:
legend

,Minimum_Score,Maximum_Score,Description
0,0,70,Poor
1,71,85,Needs Improvement
2,86,90,Adequate
3,91,100,Good


## 2. Local database

The first step in our analysis is to import the data into a local PostgreSQL database.

1. Connect to a local Postgres database and import the files to separate tables.

**Hint:** The files are probably not encoded in utf8 and this could create a problem when importing the data into postgres. You can read more about encodings here: http://www.postgresql.org/docs/current/interactive/multibyte.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('postgresql://llevin@localhost:5432/southwind')

In [34]:
def load_to_sql(name):
    df = pd.read_csv('/home/llevin/Desktop/DSI-SF-2-llevin16/Week 6 Notes & Code/Datasets/businesses.csv', encoding='latin1')
    df.to_sql(name, engine, flavor='postgres', if_exists='replace')
    print "done", name

load_to_sql('business')

done business


In [35]:
#inspections.to_sql('inspections',con=engine)
#violations.to_sql('violations',con=engine)

### 2.b Display the first few lines of each table

In [25]:
%load_ext sql

/home/llevin/anaconda2/envs/dsi/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/llevin/anaconda2/envs/dsi/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [40]:
%%sql postgresql://llevin@localhost:5432/southwind
        
select * from violations limit 10

10 rows affected.


index,business_id,date,description
0,10,20140729,Insufficient hot water or running water [ date violation corrected: 8/7/2014 ]
1,10,20140729,Unapproved or unmaintained equipment or utensils [ date violation corrected: 8/7/2014 ]
2,10,20140114,Inadequate and inaccessible handwashing facilities [ date violation corrected: 1/24/2014 ]
3,10,20140114,Unclean or degraded floors walls or ceilings [ date violation corrected: 1/24/2014 ]
4,10,20140114,Improper storage of equipment utensils or linens [ date violation corrected: 1/24/2014 ]
5,19,20141110,Improper storage of equipment utensils or linens
6,19,20141110,Inadequate food safety knowledge or lack of certified food safety manager
7,19,20140214,Inadequately cleaned or sanitized food contact surfaces [ date violation corrected: 2/14/2014 ]
8,19,20140214,Permit license or inspection report not posted [ date violation corrected: 11/10/2014 ]
9,19,20130904,Foods not protected from contamination [ date violation corrected: 9/4/2013 ]


## 2.b Investigate violations

Let's focus on the violations table initially.


Answer these questions using sql:
1. How many violations are there?
- How many businesses committing violations?
- What's the average number of violations per business?

Answer these questions using python
1. Draw a plot of the violations count
- Is the average number of violations meaningful?
- Draw a plot of the normalized cumulative violation counts. Can we discard the restaurants with few violations?
- Where would you draw a threshold if you were to keep 90% of the violations?

### 2.c Investigate Inspections

In the previous step we looked at violations count. However we also have an inspection score available in the inspections table. Let's have a look at that too.

Answer these questions using SQL:
1. What's the average score for the whole city?
1. What's the average score per business?
- Does the score correlate with the number of inspections?
- Create a dataframe from a table with the following columns:
    business_id, average_score, number_of_inspections, number_of_violations
- Use pandas to do a scatter matrix plot of average_score, number_of_inspections, number_of_violations to check for correlations

## 3 Zipcode analysis

The town administration would like to know which zip code are the ones where they should focus the inspections.

Use the information contained in the `businesses` table as well as the previous tables to answer the following questions using SQL:

1. Count the number of businesses per zipcode and sort them by descending order
- Which are the top 5 zipcodes with the worst average score?
    - restrict your analysis to the zipcodes with at least 50 businesses
    - do a simple average of the inspections scores in the postal code
- Which are the top 5 zipcodes with the highest number of violations per restaurant?
    - restrict your  analysis to the zipcodes with at least 50 businesses


## Final recommendation
Give a final recommendation on which 2 zipcodes should the administration focus and choose an appropriate plot to convince them visually.

## Bonus: Neighborhood data

Instead of looking at zipcodes we may be interested in using Neighborhood names.

It's beyond the scope of this lab to do a proper introduction to Geocoding and Reverse Geocoding, but we will give some pointers for further exploration.

### 1. Google Geocoding API
Have a look at:
- https://developers.google.com/maps/documentation/geocoding/intro
- https://maps.googleapis.com/maps/api/geocode/json?address=
- https://maps.googleapis.com/maps/api/geocode/json?latlng=

Through this API you can retrieve an address or a neighborhood from a lat-lon pair (reverse geocoding), or you can retrieve lat long and other information from an address (geocoding).

1. Try experimenting with and retrieving a few addresses
- Note that google imposes limits on the number of free queries
- How many missing lat-lon pairs do we have?

### Bonus 2
The pycurl library seems to be faster than requests in getting information from the google api.

1. See if you can extract the neighborhood from an address using the geocode api and a bit of json parsing
- Note that you would surely hit the daily limit if you pulled each address' neighborhood from the api

### Bonus 3
We can find the neighborhood using the polygons associated to each of them.
[Here](https://www.google.com/fusiontables/DataSource?docid=1zNwsvTwj-dH0QxuuDrKFsyfNklajd7WwEyaZ2U9M#rows:id=1) you can find these polygons (and we also copied them [locally](../../assets/datasets/sfneighborhoods.csv).

[This article](http://streamhacker.com/2010/03/23/python-point-in-polygon-shapely/) describes how to use the shapely package to check if a point belongs to a polygon.

- See if you can build a function that retrieves the neighborhood for a given address using the polygon data

### Further exploration

Postgres is actually GIS enabled, so we could do location based queries directly in the database.

Have a look at http://postgis.refractions.net/ for more information.